# Import dataset

In [37]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

train_set = tfds.load('imdb_reviews', split='train', as_supervised=True).take(5000)
test_set = tfds.load('imdb_reviews', split='test', as_supervised=True).take(1000)

# Prepare tokens

In [45]:
max_text_length = 0

# training
X_train = []
y_train = []
for i, j in train_set:
    i = str(i.numpy())
    max_text_length = max(max_text_length, len(i))
    X_train.append(i)
    y_train.append(int(j))

tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_text_length, padding='post')

X_train = np.array(X_train)
y_train = np.array(y_train)

# testing
X_test = []
y_test = []
for i, j in test_set:
    i = str(i.numpy())
    X_test.append(i)
    y_test.append(int(j))

X_test = tokenizer.texts_to_sequences(X_test)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_text_length, padding='post')

X_test = np.array(X_test)
y_test = np.array(y_test)

# Build the RNN
We need to use an RNN to help predict positive/negative reviews because the order of words obviously matters in sentences. An embedding layer is needed to convert sequences of tokens into sequences of vectors that can be easily understood by the RNN layer.

In [39]:
model = keras.Sequential()
model.add(keras.layers.Embedding(10001, 128, mask_zero=True))
model.add(keras.layers.SimpleRNN(100, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.BinaryAccuracy()])

# Train the model

In [40]:
epochs = 10

model.fit(X_train, y_train, epochs=epochs, validation_split=0.1, batch_size=256)

Epoch 1/10
18/18 [==============================] - 762s 42s/step - loss: 0.6913 - binary_accuracy: 0.5158 - val_loss: 0.6899 - val_binary_accuracy: 0.5360
Epoch 2/10
18/18 [==============================] - 778s 43s/step - loss: 0.6530 - binary_accuracy: 0.7062 - val_loss: 0.6512 - val_binary_accuracy: 0.6040
Epoch 3/10
18/18 [==============================] - 773s 43s/step - loss: 0.5995 - binary_accuracy: 0.7698 - val_loss: 0.6313 - val_binary_accuracy: 0.6580
Epoch 4/10
18/18 [==============================] - 784s 44s/step - loss: 4282.6270 - binary_accuracy: 0.6813 - val_loss: 0.7717 - val_binary_accuracy: 0.4900
Epoch 5/10
18/18 [==============================] - 757s 42s/step - loss: 0.7514 - binary_accuracy: 0.5140 - val_loss: 0.9542 - val_binary_accuracy: 0.4860
Epoch 6/10
18/18 [==============================] - 746s 41s/step - loss: 0.7697 - binary_accuracy: 0.5180 - val_loss: 0.8752 - val_binary_accuracy: 0.4940
Epoch 7/10
18/18 [==============================] - 753s 42s/

# Evaluate on test set

In [46]:
print(model.metrics_names)
print(model.evaluate(X_test, y_test))

['loss', 'binary_accuracy']
32/32 [==============================] - 49s 2s/step - loss: 0.6940 - binary_accuracy: 0.5480
[0.6939795613288879, 0.5479999780654907]
